In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from os import listdir
from collections import defaultdict
sns.set(style="ticks", font_scale=1.2)
plt.rc("axes.spines", top=False, right=False)

from utils import *

OUTPUT measures (4): ['output_entropy', 'output_rank_correct', 'output_logprob_correct', 'output_logprobdiff']
PROCESS measures (15): ['auc_entropy', 'layer_biggest_change_entropy', 'auc_rank_correct', 'layer_biggest_change_rank_correct', 'auc_logprob_correct', 'layer_biggest_change_logprob_correct', 'auc_logprobdiff_pos', 'auc_logprobdiff_neg', 'layer_biggest_change_logprobdiff', 'auc_boosting_pos', 'auc_boosting_neg', 'layer_argmax_boosting', 'twostage_magnitude', 'twostage_magnitude_latter34', 'twostage_layer']


# 0. Helper functions and global variables

## Helper functions for computing scalar projections (Boosting metric)

In [2]:
C_VAR = "correct"
I_VAR = "incorrect"

def get_logit_diffs(row, abs_val=False,):
    correct_logits = row[f"logits_deltas_{C_VAR}"]
    incorrect_logits = row[f"logits_deltas_{I_VAR}"]
    if pd.isna(correct_logits) or pd.isna(incorrect_logits):
        return None
    else:
        if isinstance(correct_logits, str):
            correct_logits = eval(correct_logits)
        if isinstance(incorrect_logits, str):
            incorrect_logits = eval(incorrect_logits)
        first_correct_logit = correct_logits[0]
        first_incorrect_logit = incorrect_logits[0]
        if abs_val:
            return abs(first_correct_logit) - abs(first_incorrect_logit)
        else:
            return first_correct_logit - first_incorrect_logit

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors `v1` and `v2`:
        >>> angle_between((1, 0, 0), (0, 1, 0))
        1.5707963267948966
        >>> angle_between((1, 0, 0), (1, 0, 0))
        0.0
        >>> angle_between((1, 0, 0), (-1, 0, 0))
        3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

# 1. Read and process model outputs

In [3]:
def read_model_data(task, data_dir="../../data/model_output"):
    print(f"Reading model data for the following task: {task}")
    dfs = []
    for lens in ["logit_lens", "tuned_lens"]:
        print(lens)
        files = sorted([f for f in listdir(f"{data_dir}/{lens}") if f.startswith(task)])
        for file in files:
            model = file.split("_")[-1].strip(".csv")
            if model not in N_LAYERS.keys():
                print(f"Skipping model {model}")
                continue

            df = pd.read_csv(f"{data_dir}/{lens}/{file}").replace("intuitive", "incorrect")
            
            if I_VAR == "incorrect" and "intuitive" in df.columns:
                continue

            assert df.layer_idx.nunique() == N_LAYERS[model]

            # Add column for whether the layer is the final layer.
            # This is model-dependent.
            max_layer = df["layer_idx"].max()
            df["is_final_layer"] = df["layer_idx"] == max_layer

            # Add normalized layer (between 0-1).
            df["layer_01"] = df["layer_idx"] / max_layer

            df["model"] = model
            df["lens"] = lens
            dfs.append(df)
        
    df = pd.concat(dfs).reset_index().drop(columns=["index"])
    
    # Add columns for logprob diffs.
    for metric in ["sum", "mean", "first"]:
        df[f"{metric}_logprob_diff"] = (
            df[f"{metric}_logprob_{C_VAR}"] - df[f"{metric}_logprob_{I_VAR}"]
        )

    # Add columns for logit difference and term difference.
    df["logit_diff_deltas"] = df.apply(
        lambda r: get_logit_diffs(r, abs_val=False),
        axis=1
    )
    df["term_diff_deltas"] = df.apply(
        lambda r: get_logit_diffs(r, abs_val=True),
        axis=1
    )
            
    # Add scalar projections.
    x = "term_diff_deltas"
    y = "logit_diff_deltas"
    norms = np.linalg.norm(df[[x,y]], ord=2, axis=1)
    angles = np.array([
        angle_between([x,y], [1,1])
        for (x,y) in df[[x, y]].values
    ])
    scalar_projection = np.cos(angles) * norms
    df["boosting"] = scalar_projection

    # Add column for reciprocal ranks.
    for answer_option in [C_VAR, I_VAR]:
        df[f"reciprocal_rank_{answer_option}"] = 1 / df[f"rank_{answer_option}_first_token"]
        
    # Label syllogism conditions.
    if task == "syllogism":
        df["logic_belief_consistent"] = (
            df.is_realistic & \
            (((df.syllogism_condition=="consistent")&(df.is_valid)) |
            ((df.syllogism_condition=="violate")&(~df.is_valid)))
        )

    return df

In [4]:
task_dfs = {
    task: read_model_data(task=task)
    for task in TASKS
}
task_dfs["capitals-recall"].head()

Reading model data for the following task: capitals-recall
logit_lens
tuned_lens


KeyError: "None of [Index(['term_diff_deltas', 'logit_diff_deltas'], dtype='object')] are in the [columns]"

# 2. Compute all metrics

In [ ]:
def get_index_of_biggest_change(vals, negative_change=False):
    if negative_change:
        vals = -vals
    changes = [vals[i+1] - vals[i] for i in range(0, len(vals)-1)]
    idx = np.argmax(changes)
    return idx

def area_above_0(vals):
    return sum([v for v in vals if v > 0])

def area_below_0(vals):
    return sum([abs(v) for v in vals if v < 0])

def sum_of_neg_vals(vals):
    return sum([v for v in vals if v < 0])
    
def last_neg_index(vals):
    reversed_index = next((
        i for i, val in enumerate(reversed(vals), 1) if val < 0
    ), None)
    if reversed_index is None:
        return None
    else:
        return len(vals) - reversed_index

def layer_stay_pos(vals):
    """
    Returns 0-indexed layer at which all 
    subsequent values (including the current one) are positive.
    """
    last_neg = last_neg_index(vals)
    if last_neg is None:
        # always positive
        assert all(v > 0 for v in vals)
        return 0
    elif last_neg == len(vals)-1:
        # the last layer is associated with a negative value
        return len(vals)-1
    else:
        # otherwise, add one layer beyond the last negative layer
        return last_neg + 1
    
def get_auc(vals, iv, vocab_size=None):
    if iv == "entropy":
        return vals.sum()
    elif iv == "rank_correct":
        return (vals - (1/vocab_size)).sum()
    elif iv == "logprob_correct":
        return abs(vals).sum()
    elif iv == "logprobdiff" or iv == "boosting":
        pos = area_above_0(vals)
        neg = area_below_0(vals)
        return pos, neg
    else:
        raise ValueError(f"Unrecognized independent variable: {iv}")
        
def get_twostage_metric_magnitude(logprobdiffs, final_logprobdiff):
    m = min(logprobdiffs)
    if m >= 0:
        # If the intuitive answer is never favored, then M = 0.
        M = 0
    else:
        M = min(0, final_logprobdiff) - m
    return M
        
def compute_all_metrics(df, task):
    index = ITEM_META_VAR_MAP[task]
    if "model" not in index:
        index = ["model"] + index
    print(f"Index for task={task}: {index}")
    
    # Get data corresponding to final layer for output measures.
    final_layer_df = df[df.is_final_layer].set_index(index)
    
    df = df.set_index(index)
    
    results = []
    for ind in df.index.unique():
        item_data = df.loc[ind].sort_values(by="layer_idx")
        # By construction, "model" should be the first element of `ind`
        model = ind[0]
        n_layers = N_LAYERS[model]
        # There should be one value per layer
        assert len(item_data) == n_layers
        
        # ~~~~~~~~~~~~~~~~~~~~~ (0) Initialize metadata about the stimulus.
        meta_data = {k: item_data[k].values[0] for k in TASK_META_VAR_MAP[task]}
        for i, index_var in enumerate(index):
            meta_data[index_var] = ind[i]
            
        # ~~~~~~~~~~~~~~~~~~~~~ (1) Add static metrics from single layers. ("output metrics").
        static_data = {}
        for output_iv in OLD_OUTPUT_IVS:
            # FINAL layer baseline
            new_iv = OUTPUT_IV_MAP[output_iv]
            static_data[new_iv] = final_layer_df.loc[ind][output_iv]

            # MIDPOINT layer. baseline
            new_iv = OUTPUT_IV_MAP[output_iv].replace("output_", "midpoint_")
            midpoint = int(N_LAYERS[model]/2) - 1
            static_data[new_iv] = item_data[item_data.layer_idx==midpoint].loc[ind][output_iv]

        # ~~~~~~~~~~~~~~~~~~~~~ (2) Compute trajectory-based metrics.
        process_data = {}
        
        for var_name, clean_var_name in list(OUTPUT_IV_MAP.items()) + [("boosting", "boosting")]:
            # Get base name of metric. Should be one of the following: 
            # "entropy", "rank_correct", "logprob_correct", "logprobdiff"
            iv = clean_var_name.replace("output_", "")
            
            # Add AUC measure(s).
            data = item_data.sort_values(by="layer_idx")[var_name]
            auc = get_auc(data, iv, vocab_size=get_vocab_size(model))
            if iv == "logprobdiff" or iv == "boosting":
                auc_pos, auc_neg = auc
                process_data[f"auc_{iv}_pos"] = auc_pos
                process_data[f"auc_{iv}_neg"] = auc_neg
            else:
                process_data[f"auc_{iv}"] = auc

            # Add biggest change measure.
            if iv == "boosting":
                process_data[f"layer_argmax_{iv}"] = np.argmax(data)
            else:
                layer_biggest_change = get_index_of_biggest_change(
                    data,
                    negative_change=(iv == "entropy")
                )
                process_data[f"layer_biggest_change_{iv}"] = layer_biggest_change
                
            # Add dual-processing metrics (CoM and TTD).
            # =========== CoM ("change of mind"; magnitude-based)
            final_logprobdiff = final_layer_df.loc[ind]["first_logprob_diff"]
            M = get_twostage_metric_magnitude(
                item_data["first_logprob_diff"], 
                final_logprobdiff
            )
            process_data["twostage_magnitude"] = M
            # restricted to final 3/4 layers (~NOT USED IN THE PAPER~)
            threshold = int(n_layers/4)
            M_34 = get_twostage_metric_magnitude(
                item_data["first_logprob_diff"].values[threshold:], 
                final_logprobdiff
            )
            process_data["twostage_magnitude_latter34"] = M_34
            
            # =========== TTD ("time to decision"; time-based)
            process_data["twostage_layer"] = (
                (layer_stay_pos(item_data["first_logprob_diff"])+1) / n_layers
            )
        
        # ~~~~~~~~~~~~~~~~~~~~~ Combine all data.
        res = meta_data | static_data | process_data
        results.append(res)
        
    results = pd.DataFrame(results)
    
    if task == "animals":
        print("Adding German annotations")
        # Annotate with the original German names, for compatibility with human data.
        stims = pd.read_csv("../../data/stimuli/animals.csv").set_index("exemplar")
        results["exemplar_de"] = results.exemplar.apply(
            lambda e: stims.loc[e]["exemplar_de"]
        )
    return results

In [ ]:
# Compute all model predictor metrics.
task_metrics = defaultdict(dict)
for task, df in task_dfs.items():
    print("="*80)
    print(task)
    print("="*80)
    for lens in ["logit_lens", "tuned_lens"]:
        print(lens)
        metrics = compute_all_metrics(df[df.lens==lens], task)
        metrics.to_csv(
            f"../../data/model_output/processed/{task}_metrics_{lens}.csv", index=False
        )
        task_metrics[task][lens] = metrics

# 4. Combine with human data

In [ ]:
def zscore_col(df, col, group="subject_id"):
    ppt_means = df.groupby(group)[col].mean()
    ppt_stds = df.groupby(group)[col].std()
    df[f"{col}_zscore"] = df.apply(
        lambda r: (r[col]-ppt_means.loc[r[group]]) / ppt_stds.loc[r[group]],
        axis=1
    )
    return df

def read_human_data(task):
    if task == "capitals-recall":
        trial_df = pd.read_csv(f"../../data/human/{task}_trial_labeled.csv")
    else:
        trial_df = pd.read_csv(f"../../data/human/{task}_trial.csv")
        if task == "animals":
            trial_df = trial_df.rename(columns={
                "correct": "response_correct",
                "Exemplar": "exemplar_de"
            })
            # Add z-scored RTs within participant.
            trial_df = zscore_col(trial_df, "RT", group="subject_nr")
    print(f"Read human data for task = {task}")
    print("Num rows in human data:", len(trial_df))
    return trial_df

def combine_model_human_data(model_df, trial_df, task):
    assert model_df.model.nunique()==1

    # Drop columns that are all NaN. This happens for smaller models,
    # where there is no value for e.g. intermediate_layer_124_entropy
    model_df = model_df.dropna(axis=1, how='all')

    # Grab the IVs by excluding all meta variables.
    meta_vars = TASK_META_VAR_MAP[task] + ITEM_META_VAR_MAP[task]
    ivs = [c for c in model_df.columns if not c in meta_vars]
    
    if task.startswith("capitals"):
        index = "entity"
    elif task == "syllogism":
        index = "unique_id"
    elif task == "animals":
        index = "exemplar_de"

    # Add model-derived measures to human trial-level data.
    # For capitals-recall and animals there will only be one value,
    # but for capitals-recognition and syllogism we need to take the mean 
    # across the two test conditions (correct_first and intuitive_first,
    # or the two orderings of premises in the argument)
    model_means = model_df.groupby(index)[ivs].mean().reset_index()
    trial_df = trial_df.merge(model_means, on=index)

    return trial_df


for task, lens_metrics in task_metrics.items():
    print("="*80)
    print(task)
    print("="*80)
    # Read human trial-level data.
    try:
        human_data = read_human_data(task=task)
    except FileNotFoundError:
        print(f"No human data found for {task}; skipping...")
        continue
    for lens, metrics in lens_metrics.items():
        # Combine with data from each model.
        for model in metrics.model.unique():
            print(lens, model)
            trials = combine_model_human_data(metrics[metrics.model==model], human_data, task=task)
            trials.to_csv(
                f"../../data/human_model_combined/{lens}/{task}_{model}.csv", 
                index=False
            )